# CIFAR-10 Dataset Maker

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import copy
import random
import shutil

import tensorflow as tf
import numpy as np
import pandas as pd
import PIL.Image as Image

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
images = np.concatenate((train_images, test_images))
labels = np.concatenate((train_labels, test_labels))
print(f'Train: {len(train_images)}, Test: {len(test_images)}, Total: {len(train_images)+len(test_images)}')

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10/load_data
labelnames = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
labelnames